In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Suppress ta warnings
import warnings
warnings.filterwarnings("ignore")

# Auto reload local files
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
# Make files in src/ available to notebook
import sys
if '../src' not in sys.path:
    sys.path.insert(0, '../src')

In [2]:
from datetime import datetime, timedelta

# Read SPY csv, define config
watchlist = list(pd.read_csv('../data/watchlist.csv', header=0)['symbol'])
spy_constituents = list(pd.read_csv('../../data/spy_constituents.csv', header=0)['Symbol'])
random.shuffle(spy_constituents)

# Current tickers we have in database
tickers = ["AAL","AAP","AAPL","ACN","ADP","ADX","AEE","AEP","AFL","AIV","AJG","AKAM","ALGN","AMAT","AMD","AMN","AMP","AMT","ANSS","AON","AOS","ARA","ARE","ATGE","ATUS","AVGO","BA","BBWI","BCEI","BK","BKN","BKNG","BKR","BLK","BLL","BRK.B","BRO","BTC-USD","BYM","CAAP","CAT","CBRE","CBT","CELG~","CF","CHD","CHRW","CIA","CL","CMCSA","CMI","CMS","CNC","COF","COP","COST","CPRT","CRL","CSX","CTLT","CTRA","CTSH","CUZ","CYD","DD","DEI","DHR","DIS","DOV","DOW","DPW","DPZ","DRE","DRI","DRQ","DVA","DXC","DXCM","ECL","EFX","EL","ELC","EMR","ES","EVA","EXPD","EXR","FANG","FBHS","FE","FLT","FOX","FPAC","GE","GEF","GLW","GM","GNRC","GOOG","GPC","GPM","GPS","GWW","HAL","HD","HESM","HOLX","HOV","HPE","HPQ","HQL","HRI","HRL","HUBB","HUYA","ICE","INFO","INTC","IPG","IQV","IRM","ISD","ISRG","J","JCI","JDD","JNJ","KEY","KO","KRO","LEN","LH","LHX","LNC","LRCX","LTC-USD","LYV","MA","MAXR","MC","MCR","MDLZ","MEG","MLM","MMI","MMM","MO","MPC","MPWR","MRK","MRO","MSFT","MSGS","MTB","NAVB","NEM","NFJ","NLSN","NML","NNA","NOC","NRG","NTP","NVDA","NWS","NXRT","OKE","ORA","OTIS","PAYC","PCAR","PEAK","PENN","PHM","PHX","PLYM","PNR","PRU","PWR","PYS","QCOM","RCA","REGN","RF","RJF","ROST","RSG","SCHW","SEE","SLCA","SMLP","SOR","SPH","SRL","STE","SWK","SWKS","T","TAP","TDS","TDY","TECH","TEN","THO","TMO","TNK","TPR","TRMB","TSCO","TSLA","TT","TWTR","TXN","TXT","TYL","UDR","UNF","UNH","UPS","USB","V","VCIF","VFC","VMC","VNO","VRSK","VTI","VTR","WEC","WELL","WFC","WIT","WMB","WU","XOM","XPO","YUM","YUMC","ZBH","ZBRA","ZION","ZTS"]
start_date = pd.to_datetime("2019-01-01")
end_date = datetime.now() + timedelta(hours=1)
predict_window = 30

In [3]:
# Load the data from db
from sklearn.model_selection import train_test_split

import datastore as ds
from technical_signals import TechnicalSignalSet

#ds.download_candles(tickers, start_date, end_date)
def get_train_test(tickers, start_date, end_date):
    candlesticks = ds.get_candles(tickers, start_date, end_date, interval='5min')

    Xs = []
    ys = []

    for ticker in tickers:
        try:
            technical_sigs = TechnicalSignalSet(candlesticks[ticker], predict_window)
            X, y, Xy_date = technical_sigs.to_xy()
            Xs.append(X)
            ys.append(y)
        except Exception as ex:
            print(f"Exception on {ticker}:")
            print(ex)

    X = np.concatenate(Xs, axis=0)
    y = np.concatenate(ys, axis=0)

    return train_test_split(X, y, test_size=0.05)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import gc


def round_batch_size(sample_count, approximately, leeway=None):
    """
    Round batch size to a more suitable value. This helps to avoid a
    problem where the final batch has a lot of samples, but not enough for
    a full batch, leading to many samples being thrown out.

    approximately: int, leeway: int
      decide on a chunk size around a number, with specified leeway
      (leeway defaults to `approximately // 10`).
    """
    if leeway is None:
        leeway = approximately // 10
    
    # Get the number of leftover samples if we use the suggested batch size
    best_leftover = sample_count - np.floor(sample_count / approximately) * approximately

    # Brute-force search for the value that yeilds the fewest leftovers
    # within the given leeway range.
    best_chunk_count = approximately
    for offset in range(-leeway, leeway):
        chunk_size = approximately + offset
        leftover = sample_count - np.floor(sample_count / chunk_size) * chunk_size
        if leftover < best_leftover:
            best_leftover = leftover
            best_chunk_count = chunk_size
    return best_chunk_count
            
def get_train_val_dataloaders(tickers, start, end):
    X_train, X_test, y_train, y_test = get_train_test(tickers, start, end)
    
    batch_size = round_batch_size(X_train.shape[0], 8096, leeway=200)
    n_features = X_train.shape[1]

    # Convert X, y to torch tensors
    X_train_tensor = torch.from_numpy(X_train).float()
    X_test_tensor = torch.from_numpy(X_test).float()
    y_train_tensor = torch.from_numpy(y_train.reshape(y_train.shape[0], 1)).float()
    y_test_tensor = torch.from_numpy(y_test.reshape(y_test.shape[0], 1)).float()

    print(X_train_tensor.shape)
    print('Batch size:', batch_size)

    # Generators
    training_set = TensorDataset(X_train_tensor, y_train_tensor)
    dataloader_train = DataLoader(training_set, shuffle=True, batch_size=batch_size)

    validation_set = TensorDataset(X_test_tensor, y_test_tensor)
    dataloader_val = DataLoader(validation_set, shuffle=True, batch_size=batch_size)
    
    del X_train_tensor
    del X_test_tensor
    del y_train_tensor
    del y_test_tensor
    
    return dataloader_train, dataloader_val

In [5]:
# Training the model
def train(net, criterion, optimizer, dataloader_train, dataloader_val, epochs=100, device='cuda'):
    for epoch in range(epochs):
        train_loss = 0.0

        # Training
        net.train()
        for local_batch, local_labels in dataloader_train:
            #if local_batch.shape[0] != batch_size:
            #    print(f"Wrong train batch size. Skipping batch.\nThrowing away {local_batch.shape[0]} samples.")
            #    continue
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)

            # Forward pass: Compute predicted y by passing x to the model 
            y_pred = net(local_batch)
            # Compute and print loss 
            loss = criterion(y_pred, local_labels)
            # Zero gradients, perform a backward pass, update the weights. 
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step() 
            # Update loss
            train_loss += loss.item()

        # Validation
        net.eval()
        valid_loss = 0.0
        for data, labels in dataloader_val:
            #if data.shape[0] != batch_size:
            #    continue
            data, labels = data.to(device), labels.to(device)

            target = net(data)
            loss = criterion(target,labels)
            valid_loss += loss.item()

        print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(dataloader_train)} \t\t Validation Loss: {valid_loss / len(dataloader_val)}')

In [6]:
n_outputs = 1

net = nn.Sequential(
    nn.LazyLinear(256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, n_outputs),
)

In [7]:
train_device = torch.device("cuda")

# Set device for model
net = net.to(train_device)

# Select optimizerand loss criteria
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Randomize tickers
random.shuffle(tickers)

chunk_count = 7
chunks = np.array_split(tickers, chunk_count)
for chunk_tickers in chunks:
    print(chunk_tickers)
    dataloader_train, dataloader_val = get_train_val_dataloaders(chunk_tickers, start_date, end_date)
    train(net, criterion, optimizer, dataloader_train, dataloader_val, epochs=200)
    del dataloader_train
    del dataloader_val

['TSLA' 'HPQ' 'ARE' 'AEP' 'AAPL' 'ADP' 'OKE' 'V' 'PRU' 'RJF' 'MPC' 'EFX'
 'BKN' 'EXR' 'UPS' 'INTC' 'BKNG' 'JCI' 'WIT' 'AFL' 'AMT' 'AVGO' 'ANSS'
 'NLSN' 'GM' 'LHX' 'HESM' 'DIS' 'JDD' 'FBHS' 'ISD' 'CRL' 'BRO' 'CHD'
 'TRMB']
torch.Size([1445406, 58])
Batch size: 8030
Epoch 1 		 Training Loss: 0.9972086120705578 		 Validation Loss: 0.9823698163032532
Epoch 2 		 Training Loss: 0.987365596202197 		 Validation Loss: 0.9762635290622711
Epoch 3 		 Training Loss: 0.9823555291028313 		 Validation Loss: 0.9790523052215576
Epoch 4 		 Training Loss: 0.9829383614313537 		 Validation Loss: 0.9712330102920532
Epoch 5 		 Training Loss: 0.9769775893806752 		 Validation Loss: 0.967155373096466
Epoch 6 		 Training Loss: 0.9747128937784479 		 Validation Loss: 0.9688846886157989
Epoch 7 		 Training Loss: 0.9714990658325385 		 Validation Loss: 0.9515449106693268
Epoch 8 		 Training Loss: 0.9573479989615593 		 Validation Loss: 0.9520069777965545
Epoch 9 		 Training Loss: 0.9501943608015282 		 Validation Loss: 

Epoch 95 		 Training Loss: 0.6968542867602564 		 Validation Loss: 0.720846700668335
Epoch 96 		 Training Loss: 0.6901498035172731 		 Validation Loss: 0.7130887567996979
Epoch 97 		 Training Loss: 0.6752659683398778 		 Validation Loss: 0.7341434180736541
Epoch 98 		 Training Loss: 0.6991206829060507 		 Validation Loss: 0.7255140006542206
Epoch 99 		 Training Loss: 0.6809183464524495 		 Validation Loss: 0.724031400680542
Epoch 100 		 Training Loss: 0.6922672450213142 		 Validation Loss: 0.7333631634712219
Epoch 101 		 Training Loss: 0.6971494354595795 		 Validation Loss: 0.7223145723342895
Epoch 102 		 Training Loss: 0.6777296682089073 		 Validation Loss: 0.7481079459190368
Epoch 103 		 Training Loss: 0.7274628475884706 		 Validation Loss: 0.7190742790699005
Epoch 104 		 Training Loss: 0.6747715142221082 		 Validation Loss: 0.7091256380081177
Epoch 105 		 Training Loss: 0.6728539061809772 		 Validation Loss: 0.8138749241828919
Epoch 106 		 Training Loss: 0.7946546104072866 		 Validation 

Epoch 191 		 Training Loss: 0.6313749466153139 		 Validation Loss: 0.6946754574775695
Epoch 192 		 Training Loss: 0.6487259808824866 		 Validation Loss: 0.7168311595916748
Epoch 193 		 Training Loss: 0.675157459401294 		 Validation Loss: 0.744129729270935
Epoch 194 		 Training Loss: 0.6785894497323431 		 Validation Loss: 0.6800512552261353
Epoch 195 		 Training Loss: 0.6419070481595414 		 Validation Loss: 0.6751221299171448
Epoch 196 		 Training Loss: 0.6380288485664031 		 Validation Loss: 0.6773589074611663
Epoch 197 		 Training Loss: 0.6334475548886462 		 Validation Loss: 0.6694997191429138
Epoch 198 		 Training Loss: 0.6327891458463932 		 Validation Loss: 0.6630916714668273
Epoch 199 		 Training Loss: 0.6199047828906149 		 Validation Loss: 0.662360692024231
Epoch 200 		 Training Loss: 0.6119544160300197 		 Validation Loss: 0.6865121066570282
['EVA' 'ELC' 'ZBH' 'BYM' 'PCAR' 'ROST' 'ISRG' 'DRI' 'GPC' 'HOLX' 'SPH'
 'AAP' 'EL' 'BCEI' 'ADX' 'BK' 'PLYM' 'ICE' 'HRL' 'SCHW' 'MMM' 'AOS' 'PNR

Epoch 84 		 Training Loss: 0.6348426037778457 		 Validation Loss: 0.6671542823314667
Epoch 85 		 Training Loss: 0.6233250080711312 		 Validation Loss: 0.6894032061100006
Epoch 86 		 Training Loss: 0.632627888272206 		 Validation Loss: 0.6690581142902374
Epoch 87 		 Training Loss: 0.6409851548572382 		 Validation Loss: 0.6638755202293396
Epoch 88 		 Training Loss: 0.6277548054026233 		 Validation Loss: 0.6936742663383484
Epoch 89 		 Training Loss: 0.6486781450609366 		 Validation Loss: 0.6677598655223846
Epoch 90 		 Training Loss: 0.6325305712719759 		 Validation Loss: 0.6656330153346062
Epoch 91 		 Training Loss: 0.6312589707473913 		 Validation Loss: 0.6714325845241547
Epoch 92 		 Training Loss: 0.6438807021412585 		 Validation Loss: 0.6646020486950874
Epoch 93 		 Training Loss: 0.6299462521241771 		 Validation Loss: 0.6786826476454735
Epoch 94 		 Training Loss: 0.6365643044312795 		 Validation Loss: 0.6597863882780075
Epoch 95 		 Training Loss: 0.6281863330966897 		 Validation Loss: 

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
%load_ext autoreload
%reload_ext autoreload
#%autoreload 2
import backtest as bt
from strategy import PretrainedModelStrategy, SignalModelStrategy
from technical_signals import TechnicalSignalSet
from sklearn.svm import SVR


random.shuffle(tickers)
# XXX temporary - need to rework concurrency to be suitable for CUDA
# (Must use `spawn` as opposed to `fork` based concurrency I believe - separate OS processes?)
net = net.to(torch.device('cpu'))

def predict(net):
    return lambda X:\
        net(torch.from_numpy(X).float().cpu()).detach().numpy()

def df_to_signal_set(df):
    print('dataframe')
    print(type(df))
    print(df)
    return TechnicalSignalSet(df, predict_window=predict_window)

#strategy = PretrainedModelStrategy(predict(net), df_to_signal_set, cutoff=0.95, bias=0.2)
strategy = PretrainedModelStrategy(predict(net), df_to_signal_set, cutoff=4., bias=0.)
#strategy = SignalModelStrategy(SVR(), lambda df: TechnicalSignalSet(df, predict_window=14), cutoff=1., bias=0.1)
bt.comprehensive_backtest(strategy, tickers[:3], "2022-10-11", "2025-01-01", plot=True, train_test_ratio=0.8)

In [ ]:
from predictive_model import PredictiveModel
from datetime import datetime
from model_env import ModelEnv

net = net.to(torch.device('cpu'))

def df_to_signal_set(df):
    return TechnicalSignalSet(df, predict_window=predict_window)

model = PredictiveModel(net, "TorchMATI-5min", predict_window, datetime.now())
model.save()

model_env = ModelEnv.from_model(model, 'My First Test', [{'id': 'rsi'}], None, None, model_code=f"""
import torch.nn as nn

model = nn.Sequential(
    nn.LazyLinear(256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
)
""")

ds.save_model_envs([model_env])